In [6]:
"""
Feature: icu_admission
Extract from icustays.csv
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [8]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load ICU stays
icu_stay = pd.read_csv(os.path.join(icu_path, "icustays.csv"))

# Create ICU admission flag at hadm_id level
icu_by_hadm = (
    icu_stay.groupby("hadm_id")["stay_id"]
    .nunique()  # count ICU stays per admission
    .gt(0)  # True if at least one ICU stay
    .reset_index()
    .rename(columns={"stay_id": "icu_admission"})
)

# Merge with dataset
df = df.merge(
    icu_by_hadm,
    on="hadm_id",
    how="left"
)

# Fill missing values with False
df["icu_admission"] = (
    df["icu_admission"]
    .astype("boolean")
    .fillna(False)
    .astype(bool)
)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'icu_admission' added. True count: {df['icu_admission'].sum()}")
print(f"Dataset shape: {df.shape}")


Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 55)
Feature 'icu_admission' added. True count: 69246
Dataset shape: (158020, 55)


In [9]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [10]:
# cols_to_drop = [
#  "icu_admission"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)